# Paleoclimatology : temperature reconstruction between 1000 and 2100

In [ ]:
%load_ext autoreload
%autoreload 2

## Import modules

In [ ]:
import pandas as pd
from model import PaleoModel, PaleoModel2, PaleoModel3
import numpy as np
from gibbs import Gibbs
import matplotlib.pyplot as plt
from utils import FractionalGaussianNoise, AR1, AR2, WhiteNoise

## Load data

In [ ]:
Temperatures = pd.read_csv('data/Temperatures.csv', index_col=1)['temp']
T2 = pd.read_csv('data/Temperatures2.csv', index_col=1)['temp']
Forcings = pd.read_csv('data/forcings_with_prediction.csv')
Proxies = pd.read_csv('data/Proxies.csv', index_col=1)['PCR']
V_spike = pd.read_csv('data/median_volcanic_spike.csv')['x']

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.plot(Forcings['solar'], label='Total solar irradiance')
plt.legend(fontsize='x-large')
plt.subplot(2,2,2)
plt.plot(Forcings['volcanic'], label = 'Volcanism')
plt.legend(fontsize='x-large')
plt.subplot(2,2,3)
plt.plot(Forcings['CO2_RCP_2.6'], label = 'CO2 RCP 2.6')
plt.plot(Forcings['CO2_RCP_4.5'], label = 'CO2 RCP 4.5')
plt.plot(Forcings['CO2_RCP_6.0'], label = 'CO2 RCP 6.0')
plt.plot(Forcings['CO2_RCP_8.5'], label = 'CO2 RCP 8.5')
plt.legend(fontsize='x-large')
plt.subplot(2,2,4)
plt.plot(Forcings['CO2_RCP_2.6'][1900:], label = 'CO2 RCP 2.6')
plt.plot(Forcings['CO2_RCP_4.5'][1900:], label = 'CO2 RCP 4.5')
plt.plot(Forcings['CO2_RCP_6.0'][1900:], label = 'CO2 RCP 6.0')
plt.plot(Forcings['CO2_RCP_8.5'][1900:], label = 'CO2 RCP 8.5')
plt.legend(fontsize='x-large')
plt.plot()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(Proxies)
plt.show()

In [ ]:
t1, t2, t3, t4 = 1000, 1900, 2000, 2100

## Preprocess data

In [ ]:
T2 = np.array(Temperatures.loc[Temperatures.index.isin(range(t2,t3))])
RP = np.array(Proxies[Proxies.index.isin(range(t1,t3))])
S  = np.array(Forcings['solar'].loc[Forcings.index.isin(range(t1,t4))])
V  = np.array(Forcings['volcanic'].loc[Forcings.index.isin(range(t1,t4))])
C1 = np.array(Forcings['CO2_RCP_2.6'].loc[Forcings.index.isin(range(t1,t4))])
C2 = np.array(Forcings['CO2_RCP_4.5'].loc[Forcings.index.isin(range(t1,t4))])
C3 = np.array(Forcings['CO2_RCP_6.0'].loc[Forcings.index.isin(range(t1,t4))])
C4 = np.array(Forcings['CO2_RCP_8.5'].loc[Forcings.index.isin(range(t1,t4))])

In [ ]:
c12 = (C2[-1]-C1[-1])/2
c23 = (C3[-1]-C2[-1])/2
c34 = (C4[-1]-C3[-1])/2
C1_var = c12
C2_var = min(c12, c23)
C3_var = min(c23, c34)
C4_var = c34

## Gibbs sampler

In [ ]:
m = PaleoModel3(t1, t2, t3, t4, S, V, V_spike, C4, 0, T2, RP, H_init=0.6, K_init=0.5, step_H=0.03, step_K=0.03, n_iterations_MH=1)
# m = PaleoModel(t1, t2, t3, t4, S, V, C3, T2, RP, H_init=0.5, K_init=0.5, step_H=0.03, step_K=0.03, n_iterations_MH=1)

In [ ]:
g = Gibbs(m, noise_H=AR1(), noise_K=AR1(), dashboard=False)

In [ ]:
g.dashboard # Dashboard for live plotting

In [ ]:
g.run(n=50) # run gibbs sampler

## Display results

In [ ]:
g.get_results(['alpha', 'beta', 'sigma_p', 'sigma_T', 'H', 'K'], last_n=500)

In [ ]:
g.plot_T_reconstruction(last_n=500)

In [ ]:
g.histogram('alpha', last_n=5000)

In [ ]:
g.histogram('beta', last_n=5000)

In [ ]:
g.histogram('sigma_p', last_n=5000)

In [ ]:
g.histogram('sigma_T', last_n=1200)

In [ ]:
g.ECP(0.8)

In [ ]:
g.ECP(0.95)

In [ ]:
g.RMSE()

In [ ]:
g.CRPS(last_n=1200)

In [ ]:
g.IS(a=0.95, last_n=1200)

In [ ]:
g.IS(a=0.8, last_n=1200)

## Save dataset

In [ ]:
data = g.save_to_xarray(filename='../history/model3_RCP8.5_ex.netcdf')

In [ ]:
g.model.data